In [1]:
import sklearn
import pandas as pd
import numpy as np


In [3]:
df_reviews_raw = pd.read_csv('Dataset of Malicious and Benign Webpages/Webpages_Classification_train_data.csv/Webpages_Classification_train_data.csv').drop(['Unnamed: 0'], axis=1)

In [4]:
df_reviews_raw.isna().sum()

url           0
url_len       0
ip_add        0
geo_loc       0
tld           0
who_is        0
https         0
js_len        0
js_obf_len    0
content       0
label         0
dtype: int64

In [6]:
df_reviews_raw.label.describe()

count     1200000
unique          2
top          good
freq      1172747
Name: label, dtype: object

In [7]:
# Set the minimum word count threshold
min_word_count = 60

# Group by label and filter out shorter reviews within each group
df_reviews_trimmed = df_reviews_raw.groupby('label').apply(lambda x: x[x['content'].str.split().str.len() >= min_word_count].sample(25000, random_state=42)).reset_index(drop=True)

# Display summary statistics of the label column in the trimmed DataFrame
print(df_reviews_trimmed['label'].describe())

count     50000
unique        2
top         bad
freq      25000
Name: label, dtype: object


In [8]:
# Import necessary libraries
import pandas as pd

# Load your trimmed review data into df_reviews_trimmed DataFrame

# Resample trimmed dataframe to make it uniformly distributed
df_reviews_sampled = df_reviews_trimmed.groupby('label').apply(lambda x: x.sample(2000, random_state=42)).reset_index(drop=True)

# Randomly shuffle rows for aesthetics
df_reviews = df_reviews_sampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Display summary statistics of the label column in the shuffled DataFrame
print(df_reviews['label'].describe())


count     4000
unique       2
top        bad
freq      2000
Name: label, dtype: object


In [9]:
df_reviews.head()

,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label
0,http://www.lesbian-porn-tongue.com/bikini-girls/,48,137.118.56.83,United States,com,incomplete,no,550.8,495.72,teste nudger dick porn kraut criminal dumb ban...,bad
1,http://siscvball.com/,21,34.40.33.8,United States,com,complete,no,52.5,0.00,"In 1844, (domain name system) give names for t...",good
2,http://auster.verastein.de/,27,30.31.239.84,United States,de,incomplete,no,793.8,476.28,"Roll, cardinal vision. the infrared spectrum i...",bad
3,http://www.lsfnet.org/,22,144.125.3.7,United States,org,complete,yes,73.0,0.00,Change roles on brazilian trade and opening up...,good
4,http://www.uspapermoney.com,27,184.70.138.145,Canada,com,incomplete,yes,182.0,0.00,Tyler (2008). made and. Automatic digital wint...,good


In [10]:
df_reviews[['geo_loc', 'tld','who_is','https', 'label']].describe()

,geo_loc,tld,who_is,https,label
count,4000,4000,4000,4000,4000
unique,112,90,2,2,2
top,United States,com,incomplete,no,bad
freq,1776,2627,2227,2230,2000


In [11]:
import sklearn
from sklearn.preprocessing import OrdinalEncoder

# Assuming df_reviews is your DataFrame containing the categorical variables

# Initialize OrdinalEncoder
encoder = OrdinalEncoder()

# List of columns to encode
columns_to_encode = ['geo_loc', 'tld', 'who_is', 'https', 'label']

# Fit and transform the specified columns
df_reviews[columns_to_encode] = encoder.fit_transform(df_reviews[columns_to_encode])

# Now, df_reviews contains the categorical variables encoded as ordinal numbers


In [12]:
df_reviews['url'] = df_reviews['url'].apply(lambda x: ' '.join(x.split('://')[1].strip('www.').replace('.', '/').split('/')))

# Display the DataFrame
df_reviews.head()

,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label
0,lesbian-porn-tongue com bikini-girls,48,137.118.56.83,107.0,23.0,1.0,0.0,550.8,495.72,teste nudger dick porn kraut criminal dumb ban...,0.0
1,siscvball com,21,34.40.33.8,107.0,23.0,0.0,0.0,52.5,0.00,"In 1844, (domain name system) give names for t...",1.0
2,auster verastein de,27,30.31.239.84,107.0,33.0,1.0,0.0,793.8,476.28,"Roll, cardinal vision. the infrared spectrum i...",0.0
3,lsfnet org,22,144.125.3.7,107.0,72.0,0.0,1.0,73.0,0.00,Change roles on brazilian trade and opening up...,1.0
4,uspapermoney com,27,184.70.138.145,13.0,23.0,1.0,1.0,182.0,0.00,Tyler (2008). made and. Automatic digital wint...,1.0


In [13]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [17]:
import tqdm
from tqdm import tqdm
from nltk.corpus import stopwords

# Initialize tqdm pandas
tqdm.pandas()

# Get list of stopwords
stop = stopwords.words()

# Preprocess the content column
df_reviews.content = df_reviews.content.str.replace("[^\w\s]", "").str.lower()
df_reviews.content = df_reviews.content.progress_apply(lambda x: ' '.join([item for item in x.split()
                                                                           if item not in stop]))

# Preprocess the url column
df_reviews.url = df_reviews.url.str.replace("[^\w\s]", "").str.lower()
df_reviews.url = df_reviews.url.progress_apply(lambda x: ' '.join([item for item in x.split()
                                                                   if item not in stop]))


print(df_reviews.content.head())

100%|██████████| 4000/4000 [00:00<00:00, 4613.04it/s]

0    teste nudger dick porn kraut criminal dumb ban...
1    1844, (domain system) give names work introduc...
2    roll, cardinal vision. infrared spectrum in. g...
3    change roles brazilian trade opening future of...
4    tyler (2008). made and. automatic digital wint...
Name: content, dtype: object


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer for URL
tfidf = TfidfVectorizer(
    min_df=5,
    max_df=0.95,
    max_features=8000,
    stop_words='english'
)

# Fit and transform TF-IDF Vectorizer for URL
tfidf.fit(df_reviews.url)
url_tfidf = tfidf.transform(df_reviews.url)

# Fit and transform TF-IDF Vectorizer for content
tfidf.fit(df_reviews.content)
content_tfidf = tfidf.transform(df_reviews.content)


In [19]:
from sklearn.cluster import MiniBatchKMeans

def find_optimal_clusters(data, max_k):
    # Create a range of k values from 2 to max_k
    k_list = range(2, max_k+1)

    # Initialize a list to store the sum of squared errors (SSE) for each k
    sse = []

    # Iterate over each value of k
    for k in k_list:
        # Create a MiniBatchKMeans clustering model with k clusters
        kmeans_model = MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=20)

        # Fit the model to the data and compute the inertia (SSE)
        inertia = kmeans_model.fit(data).inertia_

        # Append the inertia to the sse list
        sse.append(inertia)

    # Return the list of SSE values for each k
    return sse


In [21]:
from sklearn.cluster import MiniBatchKMeans

# MiniBatchKMeans clustering with 8 clusters
kmeans = MiniBatchKMeans(n_clusters=8, init_size=1024, batch_size=2048, random_state=20)

# Fit and predict cluster labels for TF-IDF representation of URLs
df_reviews['url_cluster'] = kmeans.fit_predict(url_tfidf)


C:\Users\ACER\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [22]:
from sklearn.cluster import MiniBatchKMeans

# MiniBatchKMeans clustering with 4 clusters
kmeans = MiniBatchKMeans(n_clusters=4, init_size=1024, batch_size=2048, random_state=20)

# Fit and predict cluster labels for TF-IDF representation of content
df_reviews['content_cluster'] = kmeans.fit_predict(content_tfidf)


In [23]:
label_true = df_reviews[df_reviews['label']==1.0]
label_false = df_reviews[df_reviews['label']==0.0]

In [46]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

# Assuming df_reviews, max_sequence_length, padded_sequences, labels, train_test_split are defined as in your code

# Split data into training and test sets
train_data, test_data, train_labels, test_labels = train_test_split(padded_sequences, labels, test_size=0.3, random_state=42)

# Building LSTM model
model = Sequential()
model.add(Embedding(input_dim=np.max(padded_sequences) + 1, output_dim=100, input_length=max_sequence_length))
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train model
batch_size = 32
epochs = 10

model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_data, test_labels))


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 7, 100)            85500     
                                                                 
 lstm_2 (LSTM)               (None, 64)                42240     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 127805 (499.24 KB)
Trainable params: 127805 (499.24 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
88/88 [==============================] - 5s 11ms/step - loss: 0.5188 - accuracy: 0.8189 - val_loss: 0.1880 - val_accuracy: 0.9533
Epoch 2/10
88/88 [==============================] - 1s 7ms/step - loss: 0.1208 - accuracy: 0.9604 - val_loss: 0.0467 - val_accuracy: 0.98

In [47]:
# Evaluate model accuracy
loss, accuracy = model.evaluate(test_data, test_labels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


38/38 [==============================] - 0s 2ms/step - loss: 0.0279 - accuracy: 0.9883
Test Loss: 0.02785775437951088
Test Accuracy: 0.9883333444595337


In [48]:

model_path = 'MODEL_LSTM.h5'

model.save(model_path)

print("Model:", model_path)


Model: MODEL_LSTM.h5


C:\Users\ACER\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
